In [ ]:
# Create_Database
import cv2, os, time

count = 0
size = 4
fn_haar = 'haarcascade_frontalface_default.xml'
fn_dir = 'converted/'
fn_name = input("Enter the Person's Name: ")
path = os.path.join(fn_dir, fn_name)
if not os.path.isdir(path):
    os.mkdir(path)

(im_width, im_height) = (112, 92)
haar_cascade = cv2.CascadeClassifier(fn_haar)
webcam = cv2.VideoCapture(0)

# Set higher resolution for webcam
webcam.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
webcam.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

print("-----------------------Taking pictures----------------------")
print("--------------------Give some expressions---------------------")

while count < 45:
    (rval, im) = webcam.read()
    if not rval:
        print("Error accessing webcam")
        break
    im = cv2.flip(im, 1, 0)
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)  # Reduce noise
    mini = cv2.resize(gray, (gray.shape[1] // size, gray.shape[0] // size))
    faces = haar_cascade.detectMultiScale(mini, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    faces = sorted(faces, key=lambda x: x[3], reverse=True)

    if faces:
        face_i = faces[0]
        (x, y, w, h) = [v * size for v in face_i]
        
        # Validate face size
        if w < 100 or h < 100:  # Minimum face dimensions
            continue

        # Expand the detected face region
        margin = 10
        x = max(0, x - margin)
        y = max(0, y - margin)
        w = min(gray.shape[1] - x, w + 2 * margin)
        h = min(gray.shape[0] - y, h + 2 * margin)

        face = gray[y:y + h, x:x + w]
        face_resize = cv2.resize(face, (im_width, im_height))
        pin = sorted([int(n[:n.find('.')]) for n in os.listdir(path) if n.split('.')[0].isdigit()] + [0])[-1] + 1
        cv2.imwrite('%s/%s.png' % (path, pin), face_resize, [cv2.IMWRITE_PNG_COMPRESSION, 0])
        cv2.rectangle(im, (x, y), (x + w, y + h), (0, 255, 0), 3)
        cv2.putText(im, fn_name, (x - 10, y - 10), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0))
        time.sleep(0.38)
        count += 1

    cv2.imshow('OpenCV', im)
    key = cv2.waitKey(10)
    if key == 27:  # ESC key to break
        break

print(f"{count} images taken and saved to {fn_name} folder in the database.")
webcam.release()
cv2.destroyAllWindows()